<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2-phase2/blob/%EC%8B%A0%EB%8F%99%ED%9C%98/model_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import 및 사용 데이터 load

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = pd.read_parquet("/content/drive/MyDrive/cp2_phase2_data/fashion_logs.parquet")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201009 entries, 0 to 1254584
Data columns (total 23 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   created_at          1201009 non-null  category
 1   session_id          1201009 non-null  int32   
 2   payment_method      1201009 non-null  category
 3   promo_amount        1201009 non-null  int32   
 4   promo_code          1201009 non-null  category
 5   shipment_fee        1201009 non-null  int32   
 6   total_amount        1201009 non-null  int32   
 7   product_id          1201009 non-null  int32   
 8   productDisplayName  1201009 non-null  category
 9   quantity            1201009 non-null  int32   
 10  item_price          1201009 non-null  int32   
 11  product_gender      1201009 non-null  category
 12  baseColor           1201009 non-null  category
 13  season              1201009 non-null  category
 14  year                1201009 non-null  int32   
 15

In [4]:
data.head()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28


In [5]:
sample_data = data.tail(100000)
sample_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
1150107,2020-08-19T22:43:26,160054,LinkAja,0,No,10000,409301,4768,ADIDAS Men's F50 ST CL White T-shirt,1,...,Summer,2011,Sports,Apparel Topwear Tshirts,62919,F,1993-05-17,Android,Jawa Barat,2020-08-19
1150108,2020-08-26T07:24:02,113499,Credit Card,7892,AZ2022,10000,240052,28630,Nike Men Pintuck Grey Track Pants,1,...,Summer,2012,Sports,Apparel Bottomwear Track Pants,488,M,2002-07-16,Android,Jawa Barat,2020-08-19
1150109,2020-10-20T08:00:20,152568,Credit Card,0,No,10000,438913,59144,Tiptopp Women Black Sandals,1,...,Fall,2012,Casual,Footwear Shoes Heels,488,M,2002-07-16,Android,Jawa Barat,2020-08-19
1150110,2020-12-14T08:45:57,162446,OVO,0,No,5000,873202,21416,Roxy Women Pink Top,1,...,Winter,2012,Casual,Apparel Topwear Tops,488,M,2002-07-16,Android,Jawa Barat,2020-08-19
1150111,2020-12-14T08:45:57,162446,OVO,0,No,5000,873202,4509,Skechers Men's Arcade Midway Beige Shoe,1,...,Winter,2015,Casual,Footwear Shoes Casual Shoes,488,M,2002-07-16,Android,Jawa Barat,2020-08-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254579,2020-11-12T18:19:20,171855,Debit Card,0,No,10000,287688,42915,French Connection Women Grey Belt,1,...,Summer,2012,Casual,Accessories Belts Belts,25121,M,1993-05-19,Android,Jawa Tengah,2020-11-12
1254581,2021-12-09T07:35:18,496378,OVO,5010,AZ2022,10000,359203,19394,ADIDAS Men Solid Blue Jacket,1,...,Summer,2011,Casual,Apparel Topwear Jackets,34829,F,2006-04-21,Android,Sulawesi Selatan,2021-12-08
1254582,2021-12-11T01:20:36,769040,Credit Card,2181,WEEKENDSERU,5000,1158159,13079,Numero Uno Men White Casual Shoes,4,...,Fall,2011,Casual,Footwear Shoes Casual Shoes,99645,F,1991-03-17,Android,Jakarta Raya,2021-12-08
1254583,2019-12-18T11:01:09,623782,Credit Card,4503,AZ2022,0,184143,11312,Scullers Men Rugged Chinos Beige Trousers,1,...,Fall,2011,Casual,Apparel Bottomwear Trousers,87319,F,2007-10-26,iOS,Jawa Tengah,2019-12-18


# 2. CB 모델 구축

## 1. 학습 데이터와 테스트 데이터 분리
- 학습과 테스트를 진행하기 위해서는 최소 2개의 데이터가 필요하므로, 데이터가 1개뿐인 고객은 제거
- ~~시간 순서로 데이터를 분할하기 위해 라이브러리 미사용~~ <br>
    → 시간이 너무 오래걸려 라이브러리 사용



In [6]:
sample_data.groupby('product_id').count()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,productDisplayName,quantity,item_price,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
product_id,,,,,,,,,,,,,,,,,,,,,
1163,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1165,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1525,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1528,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1529,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59993,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
59994,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
59996,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [7]:
# 데이터가 1개뿐인 고객 제거
data_by_customer = sample_data.groupby('product_id').count()
only1_data = data_by_customer[data_by_customer['created_at'] < 2].index
data_new = data[~data['product_id'].isin(only1_data)]
data_new

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254578,2016-09-20T09:10:48,788613,Credit Card,0,No,0,114354,28377,Idee Men Brown Sunglasses,1,...,Winter,2016,Casual,Accessories Eyewear Sunglasses,77554,F,2002-08-13,Android,Jakarta Raya,2016-09-20
1254581,2021-12-09T07:35:18,496378,OVO,5010,AZ2022,10000,359203,19394,ADIDAS Men Solid Blue Jacket,1,...,Summer,2011,Casual,Apparel Topwear Jackets,34829,F,2006-04-21,Android,Sulawesi Selatan,2021-12-08
1254582,2021-12-11T01:20:36,769040,Credit Card,2181,WEEKENDSERU,5000,1158159,13079,Numero Uno Men White Casual Shoes,4,...,Fall,2011,Casual,Footwear Shoes Casual Shoes,99645,F,1991-03-17,Android,Jakarta Raya,2021-12-08
1254583,2019-12-18T11:01:09,623782,Credit Card,4503,AZ2022,0,184143,11312,Scullers Men Rugged Chinos Beige Trousers,1,...,Fall,2011,Casual,Apparel Bottomwear Trousers,87319,F,2007-10-26,iOS,Jawa Tengah,2019-12-18


In [8]:
data_new['customer_id'].value_counts()

43202    621
29496    572
10167    528
82237    526
69740    516
        ... 
50622      1
50918      1
74885      1
62791      1
58180      1
Name: customer_id, Length: 48145, dtype: int64

In [9]:
train_data, test_data = train_test_split(data_new, test_size=0.2, random_state=2)

In [10]:
train_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
1119563,2020-12-30T11:15:08,693487,OVO,0,No,10000,394509,28243,Park Avenue Men Alpha Deo,1,...,Spring,2017,Casual,Personal Care Fragrance Deodorant,52437,F,1996-04-11,Android,Jakarta Raya,2019-05-30
88514,2017-11-28T15:15:48,291619,Debit Card,2719,XX2022,25000,517618,7108,Wildcraft Unisex Olive Green Backpack,1,...,Winter,2015,Casual,Accessories Bags Backpacks,43970,F,1982-11-28,Android,Jawa Barat,2017-07-30
1219336,2020-04-20T08:04:57,651866,Credit Card,0,No,10000,318696,25642,Fastrack Men Black Casual Watch,1,...,Winter,2016,Casual,Accessories Watches Watches,2283,F,2001-12-12,Android,Lampung,2018-12-11
521063,2018-05-23T15:02:33,370152,OVO,6570,XX2022,10000,915037,39541,Bwitch Maroon Bra,1,...,Winter,2015,Casual,Apparel Innerwear Bra,83844,M,1999-08-11,Android,Jakarta Raya,2017-11-12
130966,2021-03-15T22:28:04,369238,Debit Card,0,No,0,931916,51677,Louis Philippe Men Sunglasses LP209-C3,4,...,Winter,2016,Casual,Accessories Eyewear Sunglasses,39968,M,1999-05-17,Android,Kepulauan Riau,2017-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600904,2017-07-06T13:45:31,301804,Gopay,4847,BUYMORE,5000,1248598,37248,United Colors of Benetton Men Brown Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,1996,M,1995-09-09,Android,Jawa Barat,2016-08-20
851198,2021-10-23T06:46:41,587396,Credit Card,3177,WEEKENDSERU,10000,4548991,19328,Myntra Men Maroon Sweatshirt,3,...,Fall,2011,Casual,Apparel Topwear Sweatshirts,57731,F,1991-01-15,Android,Yogyakarta,2021-06-26
278790,2020-10-28T23:26:30,250875,Credit Card,3288,AZ2022,0,578362,11555,W Women Printed Pink Kurtas,2,...,Summer,2011,Ethnic,Apparel Topwear Kurtas,46611,F,2002-02-04,Android,Jawa Barat,2016-11-06
137864,2020-03-27T01:15:07,679930,Credit Card,0,No,10000,359460,3503,ADIDAS Men Navy Blue Sandals,1,...,Spring,2013,Casual,Footwear Sandal Sports Sandals,39267,M,1994-12-23,iOS,Bali,2017-10-28


In [11]:
test_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
1150370,2020-12-06T07:55:58,466887,Credit Card,5258,WEEKENDMANTAP,15000,434943,18057,Lino Perros Women Solid Purple Wallet,1,...,Winter,2015,Casual,Accessories Wallets Wallets,61725,M,1977-01-22,Android,Jakarta Raya,2017-08-17
256919,2020-07-29T19:55:47,125080,Debit Card,0,No,0,426066,17367,Arrow Men Formal Maroon Tie+Cufflink+Pocket sq...,1,...,Fall,2011,Formal,Accessories Accessories Accessory Gift Set,33357,F,1987-08-21,iOS,Sumatera Utara,2017-10-07
1037595,2022-07-24T01:06:38,147203,Credit Card,0,No,5000,143419,27451,Jockey Essence Beige Full-Coverage Bra 1615,1,...,Summer,2018,Casual,Apparel Innerwear Bra,63506,M,2000-01-03,Android,Jakarta Raya,2022-07-23
1182833,2022-03-10T14:35:18,83032,Credit Card,0,No,10000,532173,5214,Disney Kids Boy's White Mickey Abhoy Red Set o...,2,...,Summer,2011,Casual,Apparel Topwear Tshirts,27034,F,1989-02-20,iOS,Jakarta Raya,2020-06-10
661429,2021-06-19T01:15:54,526622,Credit Card,0,No,10000,75509,53538,Amante Women Pack of 3 Assorted Briefs PFCP13,1,...,Winter,2015,Casual,Apparel Innerwear Briefs,22570,M,1992-07-30,Android,Jawa Timur,2017-01-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042973,2021-09-19T07:13:47,15444,Gopay,3149,WEEKENDMANTAP,10000,273000,19084,W Women Block Print Off White Kurta,1,...,Fall,2011,Ethnic,Apparel Topwear Kurtas,42679,M,1982-05-16,Android,Nusa Tenggara Barat,2019-11-28
915658,2022-07-02T20:03:37,267146,OVO,0,No,0,191580,27070,Yelloe Women Shoppers Fun Brown Handbag,1,...,Summer,2012,Casual,Accessories Bags Handbags,96574,F,2001-07-03,iOS,Sulawesi Tengah,2022-06-04
955144,2020-10-12T02:09:13,463407,Gopay,4265,AZ2022,0,2083903,3546,Skechers Men's Black Lifestyle Shoe,2,...,Winter,2015,Casual,Footwear Shoes Casual Shoes,9451,F,1996-10-23,Android,Kalimantan Timur,2019-01-19
1068575,2021-05-09T21:10:36,50043,OVO,0,No,10000,2519705,1678,Reebok Men Play Hard Cotton Green T-shirt,1,...,Fall,2010,Casual,Apparel Topwear Tshirts,94219,F,1986-09-11,Android,Jawa Timur,2020-11-30


## 2. 코사인유사도 계산

In [12]:
# 사용할 feature 선별
product_data = data_new[['product_id', 'product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].drop_duplicates('product_id')
product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']] = product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].astype('str')
product_data

,product_id,product_gender,baseColor,season,year,usage,Category
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner
...,...,...,...,...,...,...,...
399536,49091,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner
407254,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis
437886,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas
440721,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes


In [13]:
product_data['features'] = product_data['product_gender']+' ' +product_data['baseColor']+' ' +product_data['season']+' ' +product_data['year']+' ' +product_data['usage']+' ' +product_data['Category']
product_data

,product_id,product_gender,baseColor,season,year,usage,Category,features
0,54728,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,Men Black Summer 2012 Casual Footwear Shoes Ca...
1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters,Men Green Fall 2011 Casual Apparel Topwear Swe...
2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags,Women Purple Summer 2012 Casual Accessories Ba...
3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters,Women Red Fall 2011 Casual Apparel Topwear Swe...
4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,Women Brown Spring 2017 Casual Personal Care L...
...,...,...,...,...,...,...,...,...
399536,49091,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,Women Brown Spring 2017 Casual Personal Care L...
407254,42430,Women,Blue,Summer,2012,Ethnic,Apparel Topwear Kurtis,Women Blue Summer 2012 Ethnic Apparel Topwear ...
437886,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas,Women Brown Summer 2012 Ethnic Apparel Topwear...
440721,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes,Men Brown Fall 2012 Casual Footwear Shoes Casu...


In [14]:
# tf-idf 벡터화
tfidf = TfidfVectorizer()
product_tfidf = tfidf.fit_transform(product_data['Category'])
product_tfidf = csr_matrix(product_tfidf.todense())
product_tfidf

<33820x187 sparse matrix of type '<class 'numpy.float64'>'
	with 103344 stored elements in Compressed Sparse Row format>

In [ ]:
# 각 상품 별 유사도 확인
pd.DataFrame(cosine_similarity(product_tfidf, product_tfidf))

# 3. 성능 평가